# Image Processing
## Finding the "counts" proportional to incident photons

<img src="media/improc.png" width="700px">

### Prof. Robert Quimby
&copy; 2020 Robert Quimby

## In this tutorial you will...

- Learn about the two forms of bias in imaging data and how to remove them
- Find out that some CCD pixels are more sensitive than others and how you can correct for this
- Hear how to use overscan, bias, and flat-field data to fully process CCD images

## Recall how CCDs work

<img src="media/ccd_amplifier.png"
alt="Figure 2.3 from Howell (2006)"
width="600px">

## Relation Between Incident Photons and ADU

For each pixel (ignoring noise for the moment), 
$$C = { (B_f + B_s + \epsilon N_{\gamma}) \over g}$$
where:
* $C =$ the "counts" in ADU recorded for a given pixel
* $B_f =$ time dependant (floating) bias (measured in $e^-$)
* $B_s =$ time independant (static) bias floating bias (measured in $e^-$)
* $\epsilon =  $ photon conversion efficiency ($e^- /$ photon incident on aperature)
* $N_{\gamma} =$ number of incident photons
* $g =  $ "gain" (in $e^- / {\rm ADU}$)

## Lets look at a raw image

In [ ]:
# set up for plotting
%matplotlib notebook
import matplotlib.pyplot as plt

# load the image data
from astropy.io import fits
image = fits.getdata('media/raw.fits')

In [ ]:
# display the image
fig, ax = plt.subplots(figsize=(7,7))
ax.imshow(image, origin='lower', cmap='gray', vmin=1700, vmax=2300);

## Overscan

* After each CCD row is read out, the serial register is clocked several more times to generate virtual pixels
* These are used to measure the time dependant bias level

[CCD Readout Demonstrator](https://rquimby.sdsu.edu/ccd.html)

## Bias Exposures

* If we read out the CCD without exposing (e.g. a zero second exposure with the shutter closed), $N_{\gamma}$ will be zero so the measured signal is just the sum of the bias terms (time dependant and time independant, to first order).
$$C = { B_f + B_s + \epsilon N_{\gamma} \over g} \rightarrow { B_f \over g} + {B_s \over g}$$
* we can subtract off the median overscan value to recover the time independant (to first order) bias.

In [ ]:
# display a bias frame
bias = fits.getdata('media/bias.fits')
fig, ax = plt.subplots(figsize=(7,7))
ax.imshow(bias, origin='lower', cmap='gray', vmin=530, vmax=550);

### Every measurement comes with noise--even bias
- You will not get identical values on two consecutive bias frames
- But you can take an average over many bias frames to find the mean bias values

## How to Construct a Master Bais Frame

Master bias frames are used to remove the static bias image

* record ~25 bias frames
* subtract off the median overscan value from each bias frame
  * (taking the median is just the easiest way to do this...)
* trim off the overscan columns
* for each pixel on the CCD, determine the median, overscan subtracted value using all the bias frames
  - (this is easy with numpy)

## What we want is (a number porportional to) $N_{\gamma}$

$$ N_{\gamma} = {gC - B_f - B_s \over \epsilon}$$

Thus we need to determine each of the other parameters
* $g$ is set in the lab and can be measured from the [photon transfer curve](https://ui.adsabs.harvard.edu/abs/2019A%26A...629A..36A/abstract)
* $C$ is the raw pixel value (ADU)
* $B_f$ can be measured from the overscan
* $B_s$ can be measured from (overscan subtracted) zero second dark exposures
* but $\epsilon$ is difficult to measure directly

* however, all we need is the relative, pixel to pixel efficiency, $f$.

$$f = {\epsilon \over \bar{\epsilon}}$$

## Correcting for pixel-to-pixel variations

Recall that the ADU value (or "counts") on a given image is given by:

$$C_{i}^k = {B_{i}^k +  \epsilon_{i}^k N_{i}^k \over g_{i}^k}$$

where $i$ is the pixel index on the CCD and $k$ is the exposure number. 

The bias can be decomposed into two parts, one time-independant and one time-depenent. So, $B_{i}^k \rightarrow B^k + B_{i}$

For CCDs with one amplifier, the same gain is applied to each pixel and its value is roughly constant over time. Thus, $g_{i}^k \rightarrow g$.

### If each pixel receives the same number of photons, then:

$$\epsilon_{i}^k N_{i}^k \rightarrow \epsilon_{i} N^k$$

where we have used the fact that pixel-to-pixel efficiencies, $\epsilon_{i}$, are constant to first order in time.

Averaging over all pixels on a given exposure:

$${1 \over M} \sum_{i} \epsilon_{i} N^k = {N^k \over M} \sum_{i} \epsilon_{i} = N^k \bar{\epsilon}$$

where $M$ is the number of pixels on the image

We can then determine the pixel-to-pixel efficiencies relative to this average:
$${g C_{i}^k} - B^k - B_{i} = \epsilon_{i} N^k$$

so

$${ {g C_{i}^k} - B^k - B_{i} \over \bar{\epsilon} N^k  } = {\epsilon_{i} N^k \over \bar{\epsilon} N^k} = f_{i}$$

which means that $\epsilon_i / f_i = {\rm constant}$ for all pixels

### This lets us determine the **relative** number of incident photons in science frames

Ratio of the photons striking the $i^{\rm th}$ and $j^{\rm th}$ pixels is:
$${ N_{i}^k \over N_{j}^k } = { g C_{i}^k - b^k - B_{i} \over g C_{j}^k - b^k - B_{j} } {f_{j} \over f_{i} }$$

## Flat Field Exposures

In [ ]:
# display a twilight flat
flat = fits.getdata('media/flat.fits')
fig, ax = plt.subplots(figsize=(7,7))
ax.imshow(flat, origin='lower', cmap='gray', vmin=32000, vmax=38000);

## Detrending data

Remove bias and pixel-to-pixel variations to determine the relative numbers of photons incident on each pixel

Process:
* remove overscan
* trim to active area
* subtract master bias
* divide by master flat